In [2]:
!pip install datasets gitpython tqdm
from tqdm.auto import tqdm
import os
import git
from pathlib import Path
from datasets import Dataset, load_dataset
import glob
import re

In [3]:
# MADLAD
# download madlad monolingual tibetan corpus, clean of english text and split by shad


def is_tibetan_char(char):
    return '\u0F00' <= char <= '\u0FFF'

def is_chinese_char(char):
    return '\u4e00' <= char <= '\u9fff'

def split_tibetan_text(text):
    # Split at Tibetan shad (།) but keep the shad with the preceding text
    sentences = re.split('(།\s*)', text)
    # Recombine splits to keep shad with sentences
    sentences = [''.join(pair) for pair in zip(sentences[::2], sentences[1::2])]
    # Filter out empty sentences and strip whitespace
    return [s.strip() for s in sentences if s.strip()]

def clean_text(text):
    result = {
        'tibetan_lines': [],
        'timestamp': None,
        'metadata': ''
    }

    # First get the clean Tibetan text as before
    current_tibetan = ''
    current_metadata = ''
    metadata_parts = []

    for char in text:
        if is_chinese_char(char):
            continue

        if is_tibetan_char(char):
            if current_metadata.strip():
                metadata_parts.append(current_metadata.strip())
                current_metadata = ''
            current_tibetan += char
        else:
            if not char.isspace() or current_metadata:
                current_metadata += char

    if current_metadata.strip():
        metadata_parts.append(current_metadata.strip())

    # Split the collected Tibetan text into lines
    if current_tibetan.strip():
        result['tibetan_lines'] = split_tibetan_text(current_tibetan.strip())

    # Join metadata parts with spaces
    result['metadata'] = ' '.join(part.strip() for part in metadata_parts if part.strip())

    return result

# Load dataset
dataset = load_dataset("allenai/madlad-400", "bo")

# Process some examples
print("Processing examples from clean data:")
examples = dataset['clean'].select(range(1))
for i, example in enumerate(tqdm(examples)):
    cleaned = clean_text(example['text'])
    print(f"\nExample {i+1}:")
    print(f"Original: {example['text'][:200]}...")
    print("\nTibetan lines:")
    for j, line in enumerate(cleaned['tibetan_lines'], 1):
        print(f"Line {j}: {line}")
    print(f"\nMetadata: {cleaned['metadata']}")
    print("-" * 80)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/143k [00:00<?, ?B/s]

madlad-400.py:   0%|          | 0.00/19.3k [00:00<?, ?B/s]

The repository for allenai/madlad-400 contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/allenai/madlad-400.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] Y


bo_clean_0000.jsonl.gz:   0%|          | 0.00/42.6M [00:00<?, ?B/s]

...

Generating clean split: 0 examples [00:00, ? examples/s]

Generating noisy split: 0 examples [00:00, ? examples/s]

Processing examples from clean data:


  0%|          | 0/1 [00:00<?, ?it/s]


Example 1:
Original: བཀོད་པ 11 – Genesis 11 – The Life of Yeshu\nOctober 28, 2021 October 25, 2021 Bro Brian.\n11:1 དེའི་སྐབས་སུ་ས་གཞི་ཚང་མར་སྐད་གཅིག་རང་ཡོད་པས། མི་རྣམས་ཀྱི་སྐད་དེ་རང་བཤད་པར་བྱེད་དོ།\n11:2 དེ་རྣམས་ཤར་ཕྱོགས...

Tibetan lines:
Line 1: བཀོད་པདེའི་སྐབས་སུ་ས་གཞི་ཚང་མར་སྐད་གཅིག་རང་ཡོད་པས།
Line 2: མི་རྣམས་ཀྱི་སྐད་དེ་རང་བཤད་པར་བྱེད་དོ།
Line 3: དེ་རྣམས་ཤར་ཕྱོགས་སུ་སོང་བ་ལས།
Line 4: བྷེེ་བྷི་ལོ་ནི་ཡ་ཞེས་པའི་ཡུལ་གྱི་ཐང་ཆེན་པོ་ཞིག་ཏུ་བསླེབས་པ་དང་།
Line 5: ཁོང་ཚོ་ས་གནས་དེར་བསྡད་དོ།
Line 6: ཁོང་ཚོས་གཅིག་གིས་གཅིག་ལ་སྨྲས་པ།
Line 7: ད་ང་ཚོས་ས་སྦག་བཟོས་ནས་མེ་ལ་སྲེག་ཏེ་མཁྲེགས་པོ་བཟོས་དགོས།
Line 8: དེ་ནས་དེ་ཚོས་ཁང་པ་བཟོས་ཕྱིར་ས་སྦག་དང་དེ་ལ་བཟུང་ཕྱིར་བྲག་ཞུན་བེད་སྤྱོད་བྱེད་དོ།
Line 9: ཁོང་ཚོས།
Line 10: ད་གྲོང་ཁྱེར་ཞིག་དང་དེའི་ནང་མཁར་མཐོ་པོ་ནམ་མཁའ་ལ་ཐུག་པ་ཞིག་བཟོ་དགོས།
Line 11: དེ་ལྟར་བཟོས་ན་ང་ཚོའི་མིང་སྐད་གྲགས་བྱུང་སྟེ་འཛམ་གླིང་ནང་ང་ཚོ་འཐོར་བར་མི་འགྱུར་ཞེས་སྨྲས་སོ།
Line 12: གཙོ་བོ་དཀོན་མཆོག་གིས་གསུངས་པ།
Line 13: ད་མི་བརྒྱུད་ཐམས་ཅད་གཅིག་ཡིན་པས་སྐད་ཡང་གཅིག་ཡོད་པ་དང་།
Line 14: དེ་ཚོས་གང་འདྲ་བ